In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

# Aspect Based Sentiment Analysis on Car Reviews
## Taking Toyota Cars as an example

In [2]:
toy_rev = pd.read_csv('csvfile.csv')
toy_rev.head()

toy_rev = toy_rev['Tweets'].to_frame()

C:\Users\sm185567\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
toy_rev[toy_rev.Tweets.isnull()].shape

(19325, 1)

In [169]:
toy_rev.iloc[4]

Tweets    When you miss your sales targets! #Mirzapur2 h...
Name: 4, dtype: object

## **Using spaCy for dependency parsing which forms the crux of aspect extraction**

In [170]:
import spacy
from tqdm import tqdm
#nlp = spacy.load('en_core_web_lg', parse=True, tag=True, entity=True)
#nlp = spacy.load('en_core_web_sm',parse=True, tag=True, entity=True)
nlp = spacy.load('en')

In [171]:
nlp

 ## **Using spaCy's awesome displacy module to show the dependency relations**

In [172]:
txt = toy_rev.iloc[4]['Tweets']
nlp(txt)

When you miss your sales targets! #Mirzapur2 https://t.co/pLafT1s2Uv

In [173]:
txt = toy_rev.iloc[4]['Tweets']
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

**from https://nlp.stanford.edu/software/dependencies_manual.pdf**
### AMOD - adjectival modifier
#### An adjectival modifier of a Noun is any adjectival phrase that serves to modify the meaning of the Noun
### ex - 'Great <--amod-- Car', 'Long <--amod-- range'

In [174]:
txt = 'Drives well has great handling'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

### ADVMOD - adverb modifier
#### An adverb modifier of a word is a (non-clausal) adverb or adverb-headed phrase that serves to modify
#### the meaning of the word
### ex - 'Drives --advmod--> well'

In [175]:
txt =  "wonderful to drive the camry "
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

### XCOMP -  open clausal complement
#### An open clausal complement (xcomp) of a verb or an adjective is a predicative or clausal complement without its own subject
### ex - 'wonderful --xcomp--> drive'


In [176]:
txt =  "not wonderful to drive the camry "
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

### NEG - self explanatory
### ex - not <--neg-- wonderful

### COMPOUND WORDS
#### Generally from a review standpoint, compound words often do not offer us sentiments per se, hence my code looks for possible compound word pairs and then checks with the aspect words extracted if it can add more detail to the extracted aspects - ex Outstanding passenger van gives *more context* than Outstanding van (which is what my code would have extracted without the compound word search) while the compound word search will identify passenger van as a compound word

**Reason for using competitor name list is to remove potential misleading aspects-sentiments, since we are interested to acquire aspect info about Toyota and not any other brand. This is because a reviewer might be comparing a Benz saying it has superior handling when compared to the car the person is reviewing and this can lead to misclassifications******

In [178]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
  

toy_rev = toy_rev.dropna()

toy_rev.head(10)



for i in toy_rev['Tweets']:
    print(i)
    i = re.sub('[@#:.]', '', i)
    word_tokens = word_tokenize(i)

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    print(filtered_sentence)

In [179]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(toy_rev['Tweets']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(toy_rev['Tweets'][x])) != 0:
        lines = str(toy_rev['Tweets'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
        print("lvd",lines)
        for i in range(len(lines)):
            lines[i] = re.sub('[@#:!]', '',lines[ i])
            lines[i] = re.sub('https.*', '',lines[ i])
            lines[i] = re.sub('co.*', '',lines[ i])
            word_tokens = word_tokenize(lines[i])
            lines[i] = ' '.join(w for w in word_tokens if not w in stop_words)
        print("skfbsek",lines)    
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])
            
    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
toy_rev['compound_nouns'] = comp_terms
toy_rev['aspect_keywords'] = aspect_terms
toy_rev['competition'] = enemy
toy_rev['competition_comp_nouns'] = ecomp_terms
toy_rev['competition_aspects'] = easpect_terms
toy_rev.head()

  0%|                                                                               | 11/97962 [00:00<16:59, 96.10it/s]

 ['RT @sanket: It may sound somewhat seditious but #Mirzapur2 sucks', ' Crushed under the burden of expectations set by Season 1', '']
skfbsek ['RT sanket It may sound somewhat seditious Mirzapur2 sucks', 'Crushed burden expectations set Season 1', '']
 ['#Mirzapur2 marathon happening in house https://t', 'co/VHx407dzUA']
skfbsek ['Mirzapur2 marathon happening house', 'co/VHx407dzUA']
 ['This match was as disappointing as Mirzapur2', ' 🤯']
skfbsek ['This match disappointing Mirzapur2', '🤯']
 ['#Dream11IPL #MIvRR #Mirzapur2']
skfbsek ['Dream11IPL MIvRR Mirzapur2']
 ['you miss your sales targets! #Mirzapur2 https://t', 'co/pLafT1s2Uv']
skfbsek ['miss sales targets Mirzapur2', 'co/pLafT1s2Uv']
 ['RT @shanalaunda: you no more regret being a part of #Mirzapur2 https://t', 'co/Uu1l4H7Eov']
skfbsek ['RT shanalaunda regret part Mirzapur2', 'co/Uu1l4H7Eov']
 ['Not at all', '']
skfbsek ['Not', '']
 ['nan']
skfbsek ['nan']
 ['Watching #Mirzapur2 Episode 8 https://t', 'co/N6tnT2ADpc']
skfbsek ['Wa

  0%|                                                                              | 36/97962 [00:00<15:51, 102.91it/s]

 ['the guy with the ponytail😻😻']
skfbsek ['guy ponytail😻😻']
 ['nan']
skfbsek ['nan']
 ['#MirzapurOnPrime #Mirzapur2 #Mirzapur… https://t', 'co/DXIZajAShv']
skfbsek ['MirzapurOnPrime Mirzapur2 Mirzapur…', 'co/DXIZajAShv']
 ['RT @one_by_two: The highlight of #Mirzapur2 is to  Lilliput (M', ' M', ' Faruqui) after such a long time', ' He rocks as Dadda Tyagi ❤️']
skfbsek ['RT one_by_two The highlight Mirzapur2 Lilliput ( M', 'M', 'Faruqui ) long time', 'He rocks Dadda Tyagi ❤️']
 ['Shukla ji ka beta zinda hai। Was it all about this #Mirzapur2 ? What a waste!']
skfbsek ['Shukla ji ka beta zinda hai। Was Mirzapur2 ? What waste']
 ['Opinion Poll:']
skfbsek ['Opinion Poll']
 ['nan']
skfbsek ['nan']
 ['Which of the two web series you had liked the most?']
skfbsek ['Which two web series liked ?']
 ['nan']
skfbsek ['nan']
 ['#Scam1992OnSonyLIV… https://t', 'co/TVjmHjKH5D']
skfbsek ['Scam1992OnSonyLIV…', 'co/TVjmHjKH5D']
 ['RT @ihansika: I AM LOVING #Mirzapur2  ! It’s fantastic! Worth the hype ! A

  0%|                                                                              | 67/97962 [00:00<13:32, 120.51it/s]

...

  2%|█▌                                                                          | 2000/97962 [00:16<12:06, 132.11it/s]

 ['#KhotikarokModi #Mirzapur2review #Mirzapur2 #mirzapu…']
skfbsek ['KhotikarokModi Mirzapur2review Mirzapur2 mirzapu…']
 ["RT @varunkapurz: My wife doesn't like me anymore!!! U k why?? Coz I have blocked her from watching #BiggBoss"]
skfbsek ["RT varunkapurz My wife n't like anymore U k ? ? Coz I blocked watching BiggBoss"]
 ['@ndtv Reminds me #Mirzapur2 ']
skfbsek ['ndtv Reminds Mirzapur2']
 ['The making of another #kingofmirzapur']
skfbsek ['The making another kingofmirzapur']
 ['RT @Showbiz_IT: is #Mirzapur2 release on #AmazonPrimeIndia? Date']
skfbsek ['RT Showbiz_IT Mirzapur2 release AmazonPrimeIndia ? Date']
 ['nan']
skfbsek ['nan']
 ['https://t', 'co/Vic8Tnc9WX']
skfbsek ['', 'co/Vic8Tnc9WX']
 ['Please No Politics 🥀🇮🇳']
skfbsek ['Please No Politics 🥀🇮🇳']
 ['@AnupriyaSPatel #Mirzapur2review #Mirzapur2 @alifazal9 😇 #Mirzapur2memes https://t', 'co/3C4sCIeOJC']
skfbsek ['AnupriyaSPatel Mirzapur2review Mirzapur2 alifazal9 😇 Mirzapur2memes', 'co/3C4sCIeOJC']
 ['RT @simrankhanna_: SHI

  2%|█▌                                                                          | 2032/97962 [00:16<12:59, 123.11it/s]

 ['Teammates the game  Friendly Fire https://t', 'co/ksDctxj7fJ']
skfbsek ['Teammates game Friendly Fire', 'co/ksDctxj7fJ']
 ['Feeling bad for all those whose name is Lalit', ' #Mirzapur2']
skfbsek ['Feeling bad whose name Lalit', 'Mirzapur2']
 ['RT @team_harshita12: Dimpy 😭']
skfbsek ['RT team_harshita12 Dimpy 😭']
 ['nan']
skfbsek ['nan']
 ['#HarshitaGaur #DimpyPandit #Mirzapur2 https://t', 'co/1FXW4aa832']
skfbsek ['HarshitaGaur DimpyPandit Mirzapur2', 'co/1FXW4aa832']
 ['After watching #Mirzapur2']
skfbsek ['After watching Mirzapur2']
 ['RT @CrypticMiind: In #Mirzapur2']
skfbsek ['RT CrypticMiind In Mirzapur2']
 ['nan']
skfbsek ['nan']
 ['Muslim Maqbool is shown as tall and virile', '']
skfbsek ['Muslim Maqbool shown tall virile', '']
 ['nan']
skfbsek ['nan']
 ['RT cos only S…']
skfbsek ['RT cos S…']
 ['RT @simrankhanna_: SHIPPPPP 🥺🥺❤️']
skfbsek ['RT simrankhanna_ SHIPPPPP 🥺🥺❤️']
 ['Al Robin &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; ']
skfbsek ['Al Robin & gt ; & gt ; & gt ; & gt ; & gt ; & 

KeyboardInterrupt: 

In [ ]:
toy_rev[toy_rev['aspect_keywords']!='[]']

## We use vaderSentiment for sentiment analysis because of it's speed and simplicity. It offers 3 types of polarity -  positive, negative and neutral. As a result we can filter all aspects which have high neutral scores hence minimizing errors caused due to wrong extraction of aspects and stopwords

In [148]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [149]:
import operator
sentiment = []
for i in range(len(toy_rev)):
    score_dict={'pos':0,'neg':0,'neu':0}
    if len(toy_rev['aspect_keywords'][i])!=0: 
        for aspects in toy_rev['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
        #score_dict['neu'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
toy_rev['sentiment'] = sentiment
toy_rev.head()

,Tweets,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment
0,RT @sanket: It may sound somewhat seditious bu...,[],[],[],[],[],NaN
1,#Mirzapur2 marathon happening in house https:/...,"[(Mirzapur2 marathon, marathon)]",[],[],[],[],NaN
2,This match was as disappointing as Mirzapur2. 🤯,[],[],[],[],[],NaN
3,#Dream11IPL #MIvRR #Mirzapur2,[],[],[],[],[],NaN
4,When you miss your sales targets! #Mirzapur2 h...,"[(sales targets, targets)]",[],[],[],[],NaN


In [150]:
toy_rev['sentiment'].value_counts()

NaN    86768
pos     9760
neg     1434
Name: sentiment, dtype: int64

In [151]:
int_sent = []
for sent in toy_rev['sentiment']:
    if sent is 'NaN':
        int_sent.append('NaN')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
toy_rev['int_sent'] = int_sent
toy_rev.head()

,Tweets,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,RT @sanket: It may sound somewhat seditious bu...,[],[],[],[],[],NaN,NaN
1,#Mirzapur2 marathon happening in house https:/...,"[(Mirzapur2 marathon, marathon)]",[],[],[],[],NaN,NaN
2,This match was as disappointing as Mirzapur2. 🤯,[],[],[],[],[],NaN,NaN
3,#Dream11IPL #MIvRR #Mirzapur2,[],[],[],[],[],NaN,NaN
4,When you miss your sales targets! #Mirzapur2 h...,"[(sales targets, targets)]",[],[],[],[],NaN,NaN


In [152]:
toy_rev.to_csv('ksns.csv')

toy_rev.to_csv('ksns_updated.csv')

In [126]:
toy_rev[toy_rev['int_sent']=='1'][['Tweets','aspect_keywords']]

,Tweets,aspect_keywords
9,#Mirzapur2 fabulous series,"[fabulous series, Mirzapur2 series]"
10,Kudos @alifazal9 @battatawada for fabulous act...,[fabulous acting]
26,RT @one_by_two: The highlight of #Mirzapur2 is...,[long time]
34,RT @one_by_two: The highlight of #Mirzapur2 is...,[long time]
35,@curiousvelocite @GabbbarSingh Will there be #...,[next part]
...,...,...
97765,Very exited for most awaited series #Mirzapur2...,"[well done, Very exited]"
97792,The storm continues with its 2nd installment. ...,[2nd installment]
97950,RT @TeaWithKanmani: List of upcoming releases ...,"[upcoming releases, faltu petition]"
97952,It's 50-50 opinion about Vijay Mallya to bring...,[bring back]


### Here we have arbitarily taken ratings greater than 3 as positive and everything else as negative

In [127]:
import math
pos = []
for i in range(len(toy_rev)):
    if not math.isnan(toy_rev['Rating'][i]):
        if int(toy_rev['Rating'][i])>3:
            pos.append('1')
        else:
            pos.append('0')
    else:
        pos.append('0')
toy_rev['Positive Review'] = pos
toy_rev.head()

KeyError: 'Rating'

In [21]:
d = {'sent':toy_rev['Positive Review'],'sent_pred':toy_rev['int_sent']}
metric_df = pd.DataFrame(data=d)
metric_df.head()

KeyError: 'Positive Review'

In [ ]:
len(metric_df.sent)

## Removing NaN values in the sentiment predictions

In [ ]:
metric_df = metric_df[metric_df.sent_pred != 'NaN']
len(metric_df.sent)

In [ ]:
from sklearn.metrics import accuracy_score,auc,f1_score,recall_score,precision_score
print('accuracy')
print(accuracy_score(metric_df.sent, metric_df.sent_pred))
print('f1 score')
print(f1_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('recall')
print(recall_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('precision')
print(precision_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))

## Possible improvements that can be made
*  Tricky situation of removing stopwords to reduce unwanted extractions of non-aspects but this can also affect spaCy's dependency parsing. Same goes with noun chunk merging as well. If someone can think of a better way to remove stopwords and still retain spaCy's dependency goodness it can greatly improve the accuracy

* This is not a ML task per se since we do more of parsing than ML. Although Bi-Directional LSTM have been very good at ABSA tasks in the past, unlike semeval tasks we do not have a fixed topic for our aspects to fall into. If someone can use the parsing aspect of the code to implement BLSTM in this case, that would be great

* better alternatives to vaderSentiment if available (unsupervised/ semi-supervised methods might be better here I think)

* The very definition of aspects can be a bit vague at times hence we do not have a valid metric to measure the aspect extraction's accuracy


## P.S This is my first Kaggle Kernel and I am fairly new to python programming as well, hence my non usage of list comprehensions and functions might be evident. I highly encourage everyone to fork my code and add your own twists to increase the accuracy of both aspect extractions and sentiment analysis.